In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import io
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Make sure we don't get any GPU errors
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [4]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label


def augment(image, label):
    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_flip_left_right(image)

    return image, label

In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(augment)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)

In [6]:
class_names = [
    "Airplane",
    "Autmobile",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]


def get_model():
    model = keras.Sequential(
        [
            layers.Input((32, 32, 3)),
            layers.Conv2D(8, 3, padding="same", activation="relu"),
            layers.Conv2D(16, 3, padding="same", activation="relu"),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation="relu"),
            layers.Dropout(0.1),
            layers.Dense(10),
        ]
    )

    return model

In [7]:
model = get_model()

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

tensorboard_callback = keras.callbacks.TensorBoard( #most simple way to use tensorboard
    log_dir="logs", histogram_freq=1,
)

model.fit(
    ds_train,
    epochs=50,
    validation_data=ds_test,
    callbacks=[tensorboard_callback],
    verbose=2,
)

Epoch 1/50
1563/1563 - 5s - loss: 1.6640 - accuracy: 0.3904 - val_loss: 1.3784 - val_accuracy: 0.5055
Epoch 2/50
1563/1563 - 5s - loss: 1.3602 - accuracy: 0.5084 - val_loss: 1.1937 - val_accuracy: 0.5756
Epoch 3/50
1563/1563 - 5s - loss: 1.2414 - accuracy: 0.5548 - val_loss: 1.0862 - val_accuracy: 0.6169
Epoch 4/50
1563/1563 - 5s - loss: 1.1777 - accuracy: 0.5784 - val_loss: 1.0615 - val_accuracy: 0.6299
Epoch 5/50
1563/1563 - 5s - loss: 1.1329 - accuracy: 0.5963 - val_loss: 1.0012 - val_accuracy: 0.6521
Epoch 6/50
1563/1563 - 5s - loss: 1.0948 - accuracy: 0.6076 - val_loss: 0.9800 - val_accuracy: 0.6594
Epoch 7/50
1563/1563 - 5s - loss: 1.0656 - accuracy: 0.6187 - val_loss: 0.9530 - val_accuracy: 0.6659
Epoch 8/50
1563/1563 - 5s - loss: 1.0446 - accuracy: 0.6246 - val_loss: 0.9055 - val_accuracy: 0.6832
Epoch 9/50
1563/1563 - 5s - loss: 1.0223 - accuracy: 0.6362 - val_loss: 0.8932 - val_accuracy: 0.6890
Epoch 10/50
1563/1563 - 5s - loss: 1.0078 - accuracy: 0.6417 - val_loss: 0.9092 - 